# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

city_data_df = pd.read_csv("output_data/city_weather.csv")
city_data_df

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,4032243,Vaini,-21.2000,-175.2000,24.09,100,20,4.12,TO,1660027970
1,2071860,Esperance,-33.8667,121.9000,11.93,68,100,8.49,AU,1660027971
2,4013679,Coahuayana Viejo,18.7333,-103.6833,25.58,87,100,1.21,MX,1660027973
3,1497337,Noril'sk,69.3535,88.2027,17.46,88,100,2.40,RU,1660027974
4,4030556,Rikitea,-23.1203,-134.9692,23.15,80,99,8.72,PF,1660027976
...,...,...,...,...,...,...,...,...,...,...
571,3625929,Tucupita,9.0622,-62.0510,22.82,99,69,1.00,VE,1660028893
572,359792,Aswān,24.0934,32.9070,37.15,19,4,6.94,EG,1660028895
573,553766,Kamenka,51.3223,42.7678,29.35,29,6,2.20,RU,1660028896
574,1524296,Esil,51.9556,66.4042,24.11,40,17,8.17,KZ,1660028897


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig =gmaps.figure(center= [0,0], zoom_level=2.45)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 23) & (city_data_df["Max Temp"] < 28) \
                                & (city_data_df["Wind Speed"] < 4.5) \
                                & (city_data_df["Cloudiness"] < 30)].dropna()

ideal_city_df

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,4032243,Vaini,-21.2000,-175.2000,24.09,100,20,4.12,TO,1660027970
15,5850554,Makakilo City,21.3469,-158.0858,25.87,70,0,3.60,US,1660027993
44,3168633,Salò,45.6086,10.5310,26.33,62,0,1.22,IT,1660028036
55,2037355,Fushun,41.8558,123.9233,27.84,62,0,4.40,CN,1660028053
89,3581194,San Ignacio,17.1561,-89.0714,24.08,94,0,0.00,BZ,1660028104
126,1489962,Tavda,58.0425,65.2726,26.87,46,8,1.94,RU,1660028163
132,5699404,Scottsbluff,41.8666,-103.6672,23.77,60,0,2.97,US,1660028172
135,934649,Cap Malheureux,-19.9842,57.6142,24.15,63,0,3.58,MU,1660028177
149,2315417,Inongo,-1.9500,18.2667,23.06,60,5,0.84,CD,1660028200
156,216404,Gbadolite,4.2833,21.0167,24.90,85,17,0.20,CD,1660028211


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_city_df

hotel_df["Hotel Name"] = ''

params = {"radius":5000, "types":"lodging", "key":g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()

    results = response['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except:
        print("Missing field/result... skipping.")

    print("---------------------------")
    

Retrieving results for Index 0: Vaini.
Closest hotel is Keleti Beach Resort.
---------------------------
Retrieving results for Index 15: Makakilo City.
Closest hotel is Marriott's Ko Olina Beach Club.
---------------------------
Retrieving results for Index 44: Salò.
Closest hotel is Park Hotel Casimiro Village - Blu Hotels.
---------------------------
Retrieving results for Index 55: Fushun.
Closest hotel is Love Theme Apartment Xinhua Branch.
---------------------------
Retrieving results for Index 89: San Ignacio.
Closest hotel is Rainforest Haven Inn.
---------------------------
Retrieving results for Index 126: Tavda.
Closest hotel is The hotel is number 1.
---------------------------
Retrieving results for Index 132: Scottsbluff.
Closest hotel is Hampton Inn & Suites Scottsbluff-Conference Center.
---------------------------
Retrieving results for Index 135: Cap Malheureux.
Closest hotel is Pereybere Hotel & Spa.
---------------------------
Retrieving results for Index 149: Inon

In [6]:
hotel_df

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,4032243,Vaini,-21.2000,-175.2000,24.09,100,20,4.12,TO,1660027970,Keleti Beach Resort
15,5850554,Makakilo City,21.3469,-158.0858,25.87,70,0,3.60,US,1660027993,Marriott's Ko Olina Beach Club
44,3168633,Salò,45.6086,10.5310,26.33,62,0,1.22,IT,1660028036,Park Hotel Casimiro Village - Blu Hotels
55,2037355,Fushun,41.8558,123.9233,27.84,62,0,4.40,CN,1660028053,Love Theme Apartment Xinhua Branch
89,3581194,San Ignacio,17.1561,-89.0714,24.08,94,0,0.00,BZ,1660028104,Rainforest Haven Inn
126,1489962,Tavda,58.0425,65.2726,26.87,46,8,1.94,RU,1660028163,The hotel is number 1
132,5699404,Scottsbluff,41.8666,-103.6672,23.77,60,0,2.97,US,1660028172,Hampton Inn & Suites Scottsbluff-Conference Ce...
135,934649,Cap Malheureux,-19.9842,57.6142,24.15,63,0,3.58,MU,1660028177,Pereybere Hotel & Spa
149,2315417,Inongo,-1.9500,18.2667,23.06,60,5,0.84,CD,1660028200,Hotel Du Lac
156,216404,Gbadolite,4.2833,21.0167,24.90,85,17,0.20,CD,1660028211,Zenic


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))